# Python для анализа данных

## Веб-скрэйпинг: скачивание файлов

*На основе блокнота Татьяны Рогович, НИУ ВШЭ*
*Автор: Ян Пиле, НИУ ВШЭ*

## Скачивание файлов

Кстати, еще одно применение скрэйпинга, о котором мы пока не поговорили - это автоматизированное скачивание файлов. Например, вы хотите скачать научные статьи по определенному ключевому слову или прайс-листы поставщика, которые он загружает на сайт в эскеле.

Давайте посмотрим, как скачивать файлы на примере pdf и заодно попробуем походить по ссылкам. Кстати, этот процесс еще часто называется spidering или crawling, потому что ваш скрипт как паучок ползет по ссылкам (отсюда и название первых поисковых роботов - spider).

Давайте попробуем сохранить англоязычные summary дисертаций, защищенных в 2019 году

Мы уже отредактировали фильтры и ссылка их запомнила. Позже сегодня посмотрим как можно самим заполнять такие поля с помощью Selenium.

https://www.hse.ru/sci/diss/?author=&chief=&year=2019&type=1&degree_type=&council=&spec=&fulltext=yes

In [5]:
import requests 
from bs4 import BeautifulSoup
from selenium import webdriver as wb
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from time import sleep
import pandas as pd
driver = wb.Firefox()

driver.implicitly_wait(2)

In [6]:
link = 'https://www.hse.ru/sci/diss/?fulltext=yes'
driver.get(link)

In [7]:
soup = BeautifulSoup(driver.page_source)
# soup.find_all('a')

In [38]:
# soup.find_all('a', {'class':'link'})

Давайте для начала поэкспериментируем с первым кандидатом.

In [8]:
print(len(soup.find_all('a')))
soup.find_all('a', {'class':'link'})[:10]

189


[<a class="link link_no-underline" href="/sci/diss/?fulltext=yes">RU</a>,
 <a class="link link_no-underline" href="/en/sci/diss/?fulltext=yes">EN</a>,
 <a class="link link_no-underline" href="/sci/diss/?fulltext=yes">RU</a>,
 <a class="link link_no-underline" href="/en/sci/diss/?fulltext=yes">EN</a>,
 <a class="link" href="https://www.hse.ru/sci/diss/432474796">Альтернативные средства расчета как предмет и средство совершения преступлений в сфере экономики</a>,
 <a class="link" href="https://www.hse.ru/org/persons/69016" target="_blank">Маркунцов Сергей Александрович</a>,
 <a class="link" href="https://www.hse.ru/sci/diss/?keyword=альтернативные средства расчета">альтернативные средства расчета</a>,
 <a class="link" href="https://www.hse.ru/sci/diss/?keyword=бонусные баллы">бонусные баллы</a>,
 <a class="link" href="https://www.hse.ru/sci/diss/?keyword=виртуальная валюта">виртуальная валюта</a>,
 <a class="link" href="https://www.hse.ru/sci/diss/?keyword=виртуальная валюта многопользов

Видим, что ссылок очень много, а нам нужны только те, которые ведут на summary. Можно поискать их по тексту ссылки.

In [35]:
for link in soup.find_all('a', text='Summary'):
    print(link.get('href'))

http://www.hse.ru/data/2021/02/20/1347487870/Немова М.И._Summary.pdf
http://www.hse.ru/data/2021/03/17/1399049533/Полыковский_summary.pdf
http://www.hse.ru/data/2021/02/25/1393118412/Tiukhtiaev_Summary (1).pdf
http://www.hse.ru/data/2021/02/26/1076625271/Дементьев В.А._Summary.pdf
http://www.hse.ru/data/2021/01/27/1109877412/Salnikova_thesis_summary.pdf
http://www.hse.ru/data/2021/02/25/1352637793/Summary.pdf
http://www.hse.ru/data/2021/02/01/1362788680/Resume_phd.pdf
http://www.hse.ru/data/2021/01/19/1343833785/Хилюта В.В._Summery.pdf
http://www.hse.ru/data/2021/01/19/1343838671/Нам К.В._Summery.pdf
http://www.hse.ru/data/2021/01/18/1361147861/_Мальцев И.В._Summary.pdf


И соберем ссылки в список.

In [36]:
links = []
for link in soup.find_all('a', text='Summary'):
    print(link)
    links.append(link.get('href'))
    
print(links)

<a class="link" data-hse-file="PDF" href="http://www.hse.ru/data/2021/02/20/1347487870/Немова М.И._Summary.pdf">Summary</a>
<a class="link" data-hse-file="PDF" href="http://www.hse.ru/data/2021/03/17/1399049533/Полыковский_summary.pdf">Summary</a>
<a class="link" data-hse-file="PDF" href="http://www.hse.ru/data/2021/02/25/1393118412/Tiukhtiaev_Summary (1).pdf">Summary</a>
<a class="link" data-hse-file="PDF" href="http://www.hse.ru/data/2021/02/26/1076625271/Дементьев В.А._Summary.pdf">Summary</a>
<a class="link" data-hse-file="PDF" href="http://www.hse.ru/data/2021/01/27/1109877412/Salnikova_thesis_summary.pdf">Summary</a>
<a class="link" data-hse-file="PDF" href="http://www.hse.ru/data/2021/02/25/1352637793/Summary.pdf">Summary</a>
<a class="link" data-hse-file="PDF" href="http://www.hse.ru/data/2021/02/01/1362788680/Resume_phd.pdf">Summary</a>
<a class="link" data-hse-file="PDF" href="http://www.hse.ru/data/2021/01/19/1343833785/Хилюта В.В._Summery.pdf">Summary</a>
<a class="link" da

Отлично. Теперь нужно придумать, откуда взять название для каждого файла. Пусть это будут фамилии авторов, давайте доберемся до них. Такую задачу мы пока не решали: будем искать тэг по тексту, а потом искать его родителя (потому что ни ячейку таблицы с именем автора, ни саму таблицу не получится уникально отсечь).

In [34]:
for author in soup.find_all('div', {'class':'p1 v'}):
    try:
        print(author.span.text)
    except:
        pass


Немова Мария Игоревна
альтернативные средства расчета, 
Полыковский Даниил Александрович
генеративная модель, 
Тюхтяев Андрей Ефимович
Штырков Сергей Анатольевич
Дементьев Владислав Александрович
избирательные объединения, 
оценки материального положения, 
Шелемех Елена Александровна
американский опцион, 
восприятие, 
Хилюта Вадим Владимирович
Хомич Владимир Михайлович
имущественные преступления, 
Нам Кирилл Вадимович
гражданское право, 
Мальцев Иван Владимирович
визуализация права, 


Достанем фамилии.

In [27]:
authors = []
for author in soup.find_all('td', text='Соискатель:'):
    print(author.parent()[1].get_text().split()[0])
    authors.append(author.parent()[1].get_text().split()[0])
    
print(authors)

Махалова
Казеев
Неклюдов
Драгой
Соколова
Коган
Тюрин
Гиринский
Дорофеева
Антипкина
['Махалова', 'Казеев', 'Неклюдов', 'Драгой', 'Соколова', 'Коган', 'Тюрин', 'Гиринский', 'Дорофеева', 'Антипкина']


Проверим, что списки действительно одинаковой длины.

In [29]:
len(links) == len(authors)

True

In [38]:
print(links)

['http://www.hse.ru/data/2021/02/20/1347487870/Немова М.И._Summary.pdf', 'http://www.hse.ru/data/2021/03/17/1399049533/Полыковский_summary.pdf', 'http://www.hse.ru/data/2021/02/25/1393118412/Tiukhtiaev_Summary (1).pdf', 'http://www.hse.ru/data/2021/02/26/1076625271/Дементьев В.А._Summary.pdf', 'http://www.hse.ru/data/2021/01/27/1109877412/Salnikova_thesis_summary.pdf', 'http://www.hse.ru/data/2021/02/25/1352637793/Summary.pdf', 'http://www.hse.ru/data/2021/02/01/1362788680/Resume_phd.pdf', 'http://www.hse.ru/data/2021/01/19/1343833785/Хилюта В.В._Summery.pdf', 'http://www.hse.ru/data/2021/01/19/1343838671/Нам К.В._Summery.pdf', 'http://www.hse.ru/data/2021/01/18/1361147861/_Мальцев И.В._Summary.pdf']


Теперь попробуем сохранить файл. У нас все файлы pdf, будем переименовывать их фамилиями автора. Кстати, если файлы разного формата, то расширение можно узнать через атрибут headers

In [39]:
requests.get(links[0]).headers['content-type']

'application/pdf'

In [41]:
# потоковое чтение файла, потому pdf может быть большим и не уместиться в памяти
summary = requests.get(links[0], stream=True) 

# на всякий случай делаем проверку, иначе получим битый файл
if summary.headers['content-type'] == 'application/pdf': 

    # wb - запись байтовой информации
    fh = open('test.pdf', 'wb') 
    
    # считываем туда "содержание" файла по ссылке
    fh.write(summary.content) 
    fh.close()

In [44]:
import os
os.getcwd()

'/Users/i.pile'

Давайте теперь еще добавим имя файла по фамилии.

In [35]:
# потоковое чтение файла, потому pdf может быть большим и не уместиться в памяти
summary = requests.get(links[0], stream=True) 

# на всякий случай делаем проверку, иначе получим битый файл
if summary.headers['content-type'] == 'application/pdf': 

    # wb - запись байтовой информации
    fh = open(f'{authors[0]}.pdf', 'wb') 
    
    # считываем туда "содержание" файла по ссылке
    fh.write(summary.content) 
    fh.close()

Давайте сохраним обработку файла в функцию и соберем уже все в цикл.

In [70]:
def get_pdf(idx):
    summary = requests.get(links[idx], stream=True) # потоковое чтение файла, потому pdf может быть большим и не уместиться в памяти
    if summary.headers['content-type'] == 'application/pdf': # на всякий случай делаем проверку, иначе получим битый файл
        s = re.findall(r'\w+',links[idx].split('/')[-1])[0]
        fh = open(f'{s}.pdf', 'wb') # wb - запись байтовой информации
        fh.write(summary.content) # считываем туда "содержание" файла по ссылке
        fh.close()

In [71]:
import re
s = re.findall(r'\w+',links[0].split('/')[-1])[0]

In [72]:
f'{s}.pdf'

'Немова.pdf'

In [41]:
link = 'https://www.hse.ru/sci/diss/?fulltext=yes'
driver.get(link)

soup = BeautifulSoup(driver.page_source)

links = []
for link in soup.find_all('a', text='Summary'):
    links.append(link.get('href'))

authors = []
for author in soup.find_all('td', text='Соискатель:'):
    authors.append(author.parent()[1].get_text().split()[0])

In [46]:
links


['http://www.hse.ru/data/2021/02/20/1347487870/Немова М.И._Summary.pdf',
 'http://www.hse.ru/data/2021/03/17/1399049533/Полыковский_summary.pdf',
 'http://www.hse.ru/data/2021/02/25/1393118412/Tiukhtiaev_Summary (1).pdf',
 'http://www.hse.ru/data/2021/02/26/1076625271/Дементьев В.А._Summary.pdf',
 'http://www.hse.ru/data/2021/01/27/1109877412/Salnikova_thesis_summary.pdf',
 'http://www.hse.ru/data/2021/02/25/1352637793/Summary.pdf',
 'http://www.hse.ru/data/2021/02/01/1362788680/Resume_phd.pdf',
 'http://www.hse.ru/data/2021/01/19/1343833785/Хилюта В.В._Summery.pdf',
 'http://www.hse.ru/data/2021/01/19/1343838671/Нам К.В._Summery.pdf',
 'http://www.hse.ru/data/2021/01/18/1361147861/_Мальцев И.В._Summary.pdf']

In [73]:
for idx in range(len(links)):
    get_pdf(idx)

Готово!